## 1. 逐步尝试并完成

导包

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

from PIL import Image, ImageEnhance
from io import BytesIO
import base64

import time
import yaml
import ddddocr
from utils import BugFree

加载配置并初始化

In [31]:
# 读取配置文件
config = yaml.load(open('config.yaml', 'r', encoding='utf-8'), Loader=yaml.FullLoader)
# https://id.scu.edu.cn/enduser/sp/sso/scdxplugin_jwt23?enterpriseId=scdx&target_url=index

loading_sleep_time = config['loading_sleep_time'] # 设置加载等待时间
jumping_wait_time = config['jumping_wait_time'] # 设置跳转等待时间
driver_path = config['driver_path'] # 设置Edge浏览器驱动路径

bugfree = BugFree(config=config['bugfree']) # 初始化BugFree实例
bugfree()

ocr = ddddocr.DdddOcr(show_ad=False) # 初始化识别器

                                              
                    _ooOoo_                   
                   o8888888o                  
                   88" . "88                  
                   (| -_- |)                  
                    O\ = /O                   
                ____/`---'\____               
              .   ' \\| |// `.                
               / \\||| : |||// \              
             / _||||| -:- |||||- \            
               | | \\\ - /// | |              
             | \_| ''\---/'' | |              
              \ .-\__ `-` ___/-. /            
           ___`. .' /--.--\ `. . __           
        ."" '< `.___\_<|>_/___.' >'"".        
       | | : `- \`.;`\ _ /`;.`/ - ` : | |     
         \ \ `-. \_ __\ /__ _/ .-` / /        
 ======`-.____`-.___\_____/___.-`____.-'======
                    `=---='                   
                                              
 .............................................
          佛祖保

In [ ]:
edge_service = EdgeService(executable_path=driver_path) # 创建EdgeService实例
driver = webdriver.Edge(service=edge_service) # 创建Edge浏览器实例

打开川大教务管理系统的登录页面，切换到统一登录

In [3]:
# 打开目标网站
driver.get('http://zhjw.scu.edu.cn/login')
# print(f'\033[1;34m[info]\033[0m 已经打开网站: {driver.title} ')
print(f'\033[1;34m[info]\033[0m 已经打开网站: {driver.title} 并等待{loading_sleep_time}s，以便加载网页内容')
time.sleep(loading_sleep_time)
# 查找并点击 title 为 "跳转统一认证" 的 <a> 标签
try:
    link = WebDriverWait(driver, jumping_wait_time).until( # 使用 XPATH 定位
        EC.presence_of_element_located((By.XPATH, '//a[@title="跳转统一认证"]'))
    )
    link.click()
    print(f'\033[1;32m[info]\033[0m 已点击 "跳转统一认证" 链接！')
except Exception as e:
    print(f'\033[1;31m[Error]\033[0m 超过{jumping_wait_time}秒，等待 "跳转统一认证" 链接加载失败：{e}')
    raise e

[info] 已经打开网站: 登录 并等待2s，以便加载网页内容
[info] 已点击 "跳转统一认证" 链接！


打开"四川大学统一身份认证"的登录页面，并登录

In [4]:
# 等待跳转后的页面加载完成
try:
    WebDriverWait(driver, jumping_wait_time).until(EC.title_contains("四川大学统一身份认证")) # 跳转后的页面标题包含 "四川大学统一身份认证"
    page_source = driver.page_source
    print(f'\033[1;32m[Success]\033[0m 成功跳转，跳转后的页面标题：{driver.title}')
except Exception as e:
    print(f'\033[1;31m[Error]\033[0m 超过{jumping_wait_time}秒，等待跳转后的页面加载失败：{e}')
    raise e

username_input = driver.find_element(By.XPATH, '//input[@placeholder="请输入学(工)号"]')
password_input = driver.find_element(By.XPATH, '//input[@placeholder="请输入密码"]')
captcha_input = driver.find_element(By.XPATH, '//input[@placeholder="请输入验证码"]')
captcha_img = driver.find_element(By.CLASS_NAME, 'captcha-img')
captcha_src = captcha_img.get_attribute('src')

captcha_base64 = captcha_src.split(',')[1] # 去掉 Base64 数据的前缀（如 "data:image/png;base64,"）
captcha_image = Image.open(BytesIO(base64.b64decode(captcha_base64))) # 解码 Base64 数据
check_code = ocr.classification(captcha_image) # 识别验证码
print(f'\033[1;32m[info]\033[0m 识别验证码为：{check_code}')

username_input.send_keys(config['username'])
password_input.send_keys(config['password'])
captcha_input.send_keys(check_code)
print(f'\033[1;32m[info]\033[0m 已填写用户名、密码和验证码！')

login_button = driver.find_element(By.CSS_SELECTOR, 'button.login-btn')
login_button.click()
print(f'\033[1;32m[info]\033[0m 已点击登录按钮！')


[Success] 成功跳转，跳转后的页面标题：四川大学统一身份认证
[info] 识别验证码为：DTNx
[info] 已填写用户名、密码和验证码！
[info] 已点击登录按钮！


打开"URP高校教学管理与服务平台（学生）"的界面，并进入到"选课界面"

In [5]:
# 等待跳转后的页面加载完成
try:
    WebDriverWait(driver, jumping_wait_time).until(EC.title_contains("URP高校教学管理与服务平台")) # "URP高校教学管理与服务平台（学生）"
    print(f'\033[1;34m[info]\033[0m 跳转成功，当前页面标题：{driver.title}')
except Exception as e:
    print(f'\033[1;31m[Error]\033[0m 超过{jumping_wait_time}秒，等待跳转后的页面加载失败：{e}')
    raise e

[info] 跳转成功，当前页面标题：URP高校教学管理与服务平台（学生）


<a href="#" class="dropdown-toggle">
					<i class="menu-icon fa fa-pencil-square-o"></i>
					<span class="menu-text">选课管理
					</span>
					<b class="arrow fa fa-angle-down"></b>
				</a>

In [19]:
# 查找按钮
try:
    menu_toggler_button = driver.find_element(By.ID, 'menu-toggler')
    
    # 检查按钮是否可点击
    if menu_toggler_button.is_enabled() and menu_toggler_button.is_displayed():
        print(f'\033[1;34m[info]\033[0m "侧边栏伸缩按钮"可点击')
        menu_toggler_button.click()
        print(f'\033[1;34m[info]\033[0m 已点击"侧边栏伸缩按钮"，将等待{loading_sleep_time}s，以便渲染网页内容')
        time.sleep(loading_sleep_time)
    else:
        print(f'\033[1;33m[warning]\033[0m "侧边栏伸缩按钮"不可点击或被隐藏')
except Exception as e:
    print(f'\033[1;31m[info]\033[0m 未找到"侧边栏伸缩按钮"或操作失败：{e}')
    raise e
    
# 查找按钮

[info] "侧边栏伸缩按钮"可点击
[info] 已点击"侧边栏伸缩按钮"，将等待2s，以便渲染网页内容


In [20]:
# 查找包含 "选课管理" 的 <span>，并定位其父元素 <a> 标签
try:
    course_management_link = driver.find_element(By.XPATH, '//span[contains(text(), "选课管理")]/parent::a')
    
    # 检查 <a> 标签是否可点击
    if course_management_link.is_enabled() and course_management_link.is_displayed():
        print(f'\033[1;34m[info]\033[0m "选课管理"链接可点击')
        course_management_link.click()
        print(f'\033[1;34m[info]\033[0m 已点击"选课管理"链接，将等待{loading_sleep_time}s，以便渲染网页内容')
        time.sleep(loading_sleep_time)
    else:
        print(f'\033[1;33m[warning]\033[0m "选课管理"链接不可点击或被隐藏')
except Exception as e:
    print(f'\033[1;31m[Error]\033[0m 未找到"选课管理"链接或操作失败：{e}')
    raise e

[info] "选课管理"链接可点击
[info] 已点击"选课管理"链接，将等待2s，以便渲染网页内容


In [21]:
# 查找 href 包含 "/student/courseSelect/courseSelect/index" 的 <a> 标签
try:
    course_select_link = driver.find_element(By.XPATH, '//a[contains(@href, "/student/courseSelect/courseSelect/index")]')
    
    # 检查链接是否可点击
    if course_select_link.is_enabled() and course_select_link.is_displayed():
        print(f'\033[1;34m[info]\033[0m "选课"链接可点击')
        course_select_link.click()
        print(f'\033[1;34m[info]\033[0m 已点击"选课"链接，将等待{loading_sleep_time}s，以便渲染网页内容')
        time.sleep(loading_sleep_time)
    else:
        print(f'\033[1;33m[warning]\033[0m "选课"链接不可点击或被隐藏')
except Exception as e:
    print(f'\033[1;31m[Error]\033[0m 未找到"选课"链接或操作失败：{e}')

[warning] "选课"链接不可点击或被隐藏


```html
<div class="alert alert-block alert-danger">
    <button type="button" class="close" data-dismiss="alert" onclick="closeaApplyModel();return false;">
        <i class="ace-icon fa fa-times"></i>
    </button>
    <i class="ace-icon fa fa-exclamation-triangle red"></i>
    对不起，当前为非选课阶段！
</div>
```

In [40]:
try:
    # 查找 class 为 "alert alert-block alert-danger" 的 <div> 标签
    alert_div = driver.find_element(By.CLASS_NAME, 'alert.alert-block.alert-danger')
    print(f'\033[1;34m[info]\033[0m 找到提示信息框')

    # 检查 <div> 的内容是否包含 "对不起，当前为非选课阶段！"
    if "对不起，当前为非选课阶段！" in alert_div.text:
        print(f'\033[1;34m[info]\033[0m 提示信息内容：\033[31m{alert_div.text.strip()}\033[0m')

        # 查找 <div> 中的关闭按钮
        close_button = alert_div.find_element(By.XPATH, './/button[@type="button" and contains(@class, "close")]')
        
        # 检查按钮是否可点击
        if close_button.is_enabled() and close_button.is_displayed():
            print(f'\033[1;34m[info]\033[0m 该提示信息有 可见并可点击的 关闭按钮')
            # 抛出异常，内容为提示信息
            e = alert_div.text.strip()
            raise Exception(f"{e} \033[33m并不是脚本出错了！\033[0m")
        else:
            print(f'\033[1;33m[warning]\033[0m 关闭按钮不可点击或被隐藏')
    else:
        print(f'\033[1;34m[info]\033[0m 提示信息框内容不匹配')
except NoSuchElementException:
    print(f'\033[1;34m[info]\033[0m 未找到提示信息框')
except ElementNotInteractableException:
    print(f'\033[1;33m[warning]\033[0m 关闭按钮不可交互')
except Exception as e:
    print(f'\033[1;31m[Error]\033[0m 发生异常：{e}')
    raise  # 重新抛出异常


[info] 找到提示信息框


[info] 提示信息内容：对不起，当前为非选课阶段！
[info] 该提示信息有 可见并可点击的 关闭按钮
[Error] 发生异常：对不起，当前为非选课阶段！ 并不是脚本出错了！


Exception: 对不起，当前为非选课阶段！ [33m并不是脚本出错了！[0m